### Check weight initialization of torch nn.linear

In [1]:
import torch
import torch.nn as nn
import random
import numpy as np

import tensorflow as tf

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

torch.use_deterministic_algorithms(True)

2023-07-12 17:22:15.367440: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 17:22:15.554020: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 17:22:22.918633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/lsf10/10.1/linux3.10-glibc2.17-x86_64/lib:/data/weirauchlab/opt/lib:/da

In [3]:
a = np.array([-3, 1, 2])
print(tf.clip_by_value(a, -1, 1))
print(torch.clamp(torch.from_numpy(a), min=-1, max=1))

tf.Tensor([-1  1  1], shape=(3,), dtype=int64)
tensor([-1,  1,  1])


In [4]:
a = np.array([])
b = np.ones((2, 3))
a = np.append(a, b)
a = np.append(a, b, axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [3]:
a.shape

(12,)

In [5]:
a = torch.from_numpy(np.zeros((3, 4))).type(torch.float32)
b = torch.normal(0.0, 1.0, size=(3, 4), dtype=torch.float32)
print(a)
print(b)
print(a*b)
print(torch.matmul(a, b.T))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[ 0.2815,  0.0562,  0.5227, -0.2384],
        [-0.0499,  0.5263, -0.0085,  0.7291],
        [ 0.1331,  0.8640, -1.0157, -0.8887]])
tensor([[0., 0., 0., -0.],
        [-0., 0., -0., 0.],
        [0., 0., -0., -0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [22]:
N = 10000

In [3]:
a = torch.randn(3, 10)
print(a.numpy())

[[ 1.9269153   1.4872841   0.9007172  -2.105521    0.67841846 -1.2345449
  -0.04306748 -1.604667   -0.7521353   1.648723  ]
 [-0.39247864 -1.4036071  -0.7278813  -0.5594302  -2.3169227  -0.21680473
  -1.3846737  -0.87123615 -0.22336592  1.7173615 ]
 [ 0.31888032 -0.42451897 -0.8285543   0.33089843 -1.5575725   0.9956361
  -0.87978584 -0.60114205 -1.2741512   2.122785  ]]


In [35]:
W = torch.normal(mean=0.01, std=1.0, size=(10, 5), requires_grad=True)
b = torch.normal(mean=0.01, std=1.0, size=(5, 1), requires_grad=True)

In [36]:
class default_linear(nn.Module):
    def __init__(self):
        super().__init__()
        self.W = nn.Linear(
            in_features=10,
            out_features=5,
            bias=True
        )
        self.W.weight.data = W
        self.W.bias.data = b
        #torch.nn.init.normal_(self.W.weight, mean=0.01, std=1.0)
    
    def forward(self, x):
        return self.W(x)

data = []
for i in range(N):
    model = default_linear()
    result = model(a).detach().numpy()
    data.append(result)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [34]:
class custom_linear(nn.Module):

    def __init__(self):
        super().__init__()
        self.W = torch.normal(mean=0.01, std=1.0, size=(10, 5), requires_grad=True)
        self.b = b
        
    def forward(self, x):
        return torch.matmul(x, self.W) + self.b.view(1, -1)

data_custom = []
for i in range(N):
    model_custom = custom_linear()
    res_custom = model_custom(a).detach().numpy()
    data_custom.append(res_custom)

TypeError: matmul(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [25]:
data_arr = np.stack(data)   # shape = (1000, 3, 5)
data_arr_cus = np.stack(data_custom)    # shape = (1000, 3, 5)

### Draw the distribution of the two vectors

In [7]:
import matplotlib.pyplot as plt

In [26]:
arr_mean = np.mean(data_arr, axis=0)
arr_cus_mean = np.mean(data_arr_cus, axis=0)

As a sanity check, make sure the mean within 2 subsamples of a model is similar

In [27]:
arr_1, arr_2 = data_arr[:5000, ...], data_arr[5000:, ...]
arr1mean, arr2mean = np.mean(arr_1, axis=0), np.mean(arr_2, axis=0)

### Comparison between Tensorflow and Pytorch functions

In [2]:
a = np.array([[1, 2, 3], [1, 2, 4]])
tf.pad(a, [[0, 5], [0, 0]])

2023-06-27 00:59:11.096249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/lsf10/10.1/linux3.10-glibc2.17-x86_64/lib:/data/weirauchlab/opt/lib:/data/weirauchlab/opt/lib64:/data/weirauchlab/local/lib
2023-06-27 00:59:11.096382: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-27 00:59:11.096437: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bmiclusterp2.chmcres.cchmc.org): /proc/driver/nvidia/version does not exist
2023-06-27 00:59:11.099006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operation

<tf.Tensor: shape=(7, 3), dtype=int64, numpy=
array([[1, 2, 3],
       [1, 2, 4],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])>

In [3]:
a = torch.tensor([[1, 2, 3], [1, 2, 4]])
torch.nn.functional.pad(a, (0, 0, 0, 5))

tensor([[1, 2, 3],
        [1, 2, 4],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [18]:
class A(nn.Module):
    def __init__(self):
        super().__init__()
        self.l = nn.Linear(5, 10)
        self.build()

    def build(self):
        self.b = nn.Parameter(torch.ones(3, 3))

    def forward(self, x):
        return x+self.b
    
model = A()

In [27]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
Parameter containing:
tensor([[-0.0772,  0.0934,  0.2309,  0.3610,  0.4074],
        [-0.3546,  0.1125, -0.1924, -0.0490, -0.3347],
        [ 0.4073, -0.3282,  0.2390,  0.1572,  0.1453],
        [-0.2418,  0.4065,  0.0983,  0.0575, -0.3941],
        [ 0.1877, -0.0671, -0.2049,  0.3841,  0.0997],
        [-0.2474, -0.2264, -0.0214,  0.2497, -0.1143],
        [-0.2552, -0.1531, -0.3341,  0.1595,  0.3462],
        [-0.4210,  0.1039,  0.2310,  0.0811, -0.1593],
        [ 0.2334,  0.2350,  0.1672, -0.0786, -0.1184],
        [ 0.0478, -0.0790, -0.1333,  0.2859,  0.3843]], requires_grad=True)
Parameter containing:
tensor([-0.0443, -0.1001,  0.0065, -0.0267,  0.1075,  0.1253, -0.4062, -0.1650,
         0.3766,  0.1742], requires_grad=True)


### Find out why pytorch model returns a tuple

In [7]:
import torch
import torch.nn as nn
import numpy as np

from cellbox.config import Config
from cellbox.model_torch import CellBox

In [2]:
class BaseModel(nn.Module):

    def __init__(self, args):
        super().__init__()
        self.args = args
        self.build()

    def build(self):
        raise NotImplementedError
    

class TempModel(BaseModel):
    def build(self):
        self.a = nn.Linear(5, 5)
        self.b = nn.Linear(5, 5)

    def forward(self, inputs):
        return self.b(self.a(inputs))

In [8]:
conf = Config("/users/ngun7t/Documents/cellbox-jun-6/notebooks/results/Example_RP_d2f1204ae85ed795160c7d3209f04b32/config.json")

In [9]:
args = {"a": 1, "b": 2}
a = TempModel(conf)

In [10]:
print(a)

TempModel(
  (a): Linear(in_features=5, out_features=5, bias=True)
  (b): Linear(in_features=5, out_features=5, bias=True)
)


In [ ]:
actual = CellBox()